# Instructions for Freesurfer

In [ ]:
import os 
import shutil
import glob

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
stats1 = pd.read_csv('/mnt/data/processed_data/freesurfer_stats/aseg_stats_run1.txt', sep ='\t')
stats1


In [ ]:
stats2 = pd.read_csv('/mnt/data/processed_data/freesurfer_stats/aseg_stats_run2.txt', sep ='\t')
stats2

In [ ]:
stats3 = pd.read_csv('/mnt/data/processed_data/freesurfer_stats/aseg_stats_run3.txt', sep ='\t')
stats3

In [ ]:
total_stats = pd.concat([stats3, stats2, stats1])

In [ ]:
#total_stats.to_csv('../../secret_data/brain_volumes_from_freesurfer_no_qc.csv')

In [ ]:
total_stats.columns

In [ ]:
total_stats = total_stats.rename(columns= {'Measure:volume':'Participant ID'})
total_stats['Participant ID'] = total_stats['Participant ID'].str.replace('/', '')

In [ ]:
total_stats.head(4)

In [ ]:
total_stats.to_csv('/mnt/data/processed_data/freesurfer_stats/brain_volumes_from_freesurfer_no_qc.csv')

In [ ]:
total_stats.tail(3)

In [ ]:
#total

In [ ]:
total_stats_numeric = total_stats.drop(['Participant ID'], axis=1)
total_stats_numeric.corr() 
#.corr()

In [ ]:
sns.heatmap(total_stats_numeric.corr(method='spearman', numeric_only=False))

Let's ask if left and right correlate...or other things.

In [ ]:
left_to_right = total_stats_numeric[['Left-Lateral-Ventricle', 'Left-Inf-Lat-Vent',
       'Left-Cerebellum-White-Matter', 'Left-Cerebellum-Cortex',
       'Left-Thalamus-Proper', 'Left-Caudate', 'Left-Putamen', 'Left-Pallidum',
        'Left-Hippocampus',
       'Left-Amygdala', 'Left-Accumbens-area', 'Left-VentralDC',
       'Left-vessel', 'Left-choroid-plexus', 'Right-Lateral-Ventricle',
       'Right-Inf-Lat-Vent', 'Right-Cerebellum-White-Matter',
       'Right-Cerebellum-Cortex', 'Right-Thalamus-Proper', 'Right-Caudate',
       'Right-Putamen', 'Right-Pallidum', 'Right-Hippocampus',
       'Right-Amygdala', 'Right-Accumbens-area', 'Right-VentralDC',
       'Right-vessel', 'Right-choroid-plexus', 
        'Left-WM-hypointensities',
       'Right-WM-hypointensities', 
       'Left-non-WM-hypointensities', 'Right-non-WM-hypointensities',]]
sns.heatmap(left_to_right.corr(method='spearman', numeric_only=False))

Yes, left and right correlate. Now let's ask whether wmh increase with age....

In [ ]:
total_stats_numeric.columns

In [ ]:
total_stats

In [ ]:
wmh_info = total_stats[[ 'Participant ID','3rd-Ventricle', '4th-Ventricle', 'Brain-Stem', 'Left-Hippocampus',
       'Left-Amygdala', 'CSF',  'Right-Lateral-Ventricle',
       'Right-Inf-Lat-Vent', 'Right-Cerebellum-White-Matter',
       'Right-Cerebellum-Cortex', 'Right-Thalamus-Proper', 'Right-Caudate',
       'Right-Putamen',  'Right-Hippocampus',
       'Right-Amygdala',
       'Right-choroid-plexus',
       'WM-hypointensities', ]]

In [ ]:

wmh_info['Participant ID'] = wmh_info['Participant ID'].astype(str)
wmh_info['Participant ID'] = wmh_info['Participant ID'].str.replace("-", "_")
wmh_info

In [ ]:
for_sheet_names = pd.ExcelFile("../../secret_data/BRICK_datums_scans_clean_17072024_versie_2.xlsx")
sheet_names = for_sheet_names.sheet_names
print(sheet_names)

In [ ]:
demo_kit = '../../secret_data/BRICK_datums_scans_clean_17072024_versie_2.xlsx'
demography = pd.read_excel('../../secret_data/BRICK_datums_scans_clean_17072024_versie_2.xlsx', sheet_name='Dag van MRI+NPO overzicht ')
demography['Participant ID'] = demography['BRICK-nummer']
demography = demography[['Participant ID','Genotype', 'Leeftijd_bij_scan']]
demography

In [ ]:
fused_demo_wh = demography.merge(wmh_info, on='Participant ID')
fused_demo_wh.head(3)

In [ ]:
# Do wm-hypointensities increase with age or genotype?
simple_question = fused_demo_wh[['Genotype', 'Leeftijd_bij_scan', 'WM-hypointensities']]
# recode genotype to number
simple_question['Genotype_code'] = simple_question['Genotype'].astype('category').cat.codes
simple_question.to_csv('../../secret_data/forshow1.csv')


In [ ]:
simple_question_matrix= simple_question[['Genotype_code', 'Leeftijd_bij_scan', 'WM-hypointensities']]
simple_question_matrix.corr(method='spearman').to_csv('../../secret_data/forshow2.csv')
#simple_question_matrix.corr(method='spearman')

In [ ]:
# OK, BUT DID IT matter for neuropsychological stuff?

In [ ]:
WISC_V_BRICK_T0dd08042024 = pd.read_csv("../../secret_data/WISCV_export_gemstracker_BRICK_T0_100424.csv", sep=",")

In [ ]:
WISC_V_BRICK_T0dd08042024.head()

In [ ]:
# for f in WISC_V_BRICK_T0dd08042024.columns:
#     print(f)

In [ ]:
WISC_V_BRICK_T0dd08042024_for_castor = WISC_V_BRICK_T0dd08042024.rename(
    columns={"gr2o_patient_nr": "Participant Id",
             "DatumWISCV": "Datum_WISC_V",
            "StartWISCV": "Start_WISC_V",
            "StopWISCV": "Stop_WISC_V",
            "WISCVVolt": "WISC_V_voltooid",
            "VolgordeWISC":"Volgorde_NPO_3",
            "AfnemerWISCV":"Afnemer_WISC_V",
            "WISCVOpm": "Opmerkingen_WISC_V",
            "WISCVOpmUit":"Uitleg_Opmerkingen_WISC_V",} )


 # colnames(WISCV_gemstracker)[colnames(WISCV_gemstracker) == "gr2o_patient_nr"] <- "Participant Id"
 # colnames(WISCV_gemstracker)[colnames(WISCV_gemstracker) == "DatumWISCV"] <- "Datum_WISC_V"
 # colnames(WISCV_gemstracker)[colnames(WISCV_gemstracker) == "StartWISCV"] <- "Start_WISC_V"
 # colnames(WISCV_gemstracker)[colnames(WISCV_gemstracker) == "StopWISCV"] <- "Stop_WISC_V"
 # colnames(WISCV_gemstracker)[colnames(WISCV_gemstracker) == "WISCVVolt"] <- "WISC_V_voltooid"
 # colnames(WISCV_gemstracker)[colnames(WISCV_gemstracker) == "VolgordeWISC"] <- "Volgorde_NPO_3"
 # colnames(WISCV_gemstracker)[colnames(WISCV_gemstracker) == "AfnemerWISCV"] <- "Afnemer_WISC_V"
 # colnames(WISCV_gemstracker)[colnames(WISCV_gemstracker) == "WISCVOpm"] <- "Opmerkingen_WISC_V"
 # colnames(WISCV_gemstracker)[colnames(WISCV_gemstracker) == "WISCVOpmUit"] <- "Uitleg_Opmerkingen_WISC_V"

In [ ]:
WISC_V_BRICK_T0dd08042024_for_castor['BRICK_of_uitgebreid'] = 1
WISC_V_BRICK_T0dd08042024_for_castor.head(3)

In [ ]:
len(WISC_V_BRICK_T0dd08042024_for_castor)

In [ ]:
len(WISC_V_BRICK_T0dd08042024_for_castor['Participant Id'].unique())

In [ ]:
scores_from_wisc = WISC_V_BRICK_T0dd08042024_for_castor[[
    'Participant Id',
 'WgITot',
    'VsITot',
    'KRITot',
    'AWITot',
    'NVITot',
    'AVITot',
    'CCITot',
    'Tot',]]

In [ ]:
scores_from_wisc = scores_from_wisc.rename(columns={'Participant Id': 'Participant ID'})

In [ ]:
pd.merge?

In [ ]:
scores_from_wisc.columns

In [ ]:
#big_fuse = fused_demo_wh.merge(scores_from_wisc, on='Participant ID')
big_fuse = pd.merge(fused_demo_wh,scores_from_wisc, left_on='Participant ID', right_on='Participant ID', how='outer')
big_fuse.head(3)

In [ ]:
big_fuse_matrix = big_fuse[['Genotype','Leeftijd_bij_scan', 'Brain-Stem', 'Right-Hippocampus','Left-Hippocampus', 'Left-Amygdala',
        'Right-Lateral-Ventricle', 'Right-Inf-Lat-Vent',
       'Right-Cerebellum-White-Matter', 'Right-Cerebellum-Cortex',
       'Right-Thalamus-Proper', 'Right-Caudate', 'Right-Putamen',
        'Right-Amygdala', 'Right-choroid-plexus',
       'WM-hypointensities', 'WgITot', 'VsITot', 'KRITot', 'AWITot', 'NVITot',
       'AVITot', 'CCITot', 'Tot']]
big_fuse_matrix['Genotype_code'] = big_fuse_matrix['Genotype'].astype('category').cat.codes
big_fuse_matrix = big_fuse_matrix.drop(['Genotype'], axis=1)
big_fuse_matrix.corr(method='spearman')#.to_csv('../../secret_data/forshow3.csv')

1. first use python to move files a few at a time.

In [ ]:
find /mnt/data/output -iwholename '*Freesurfer*/*.nii.gz' -type f

In [ ]:
shutil.copy?

In [ ]:
glob.glob?

In [ ]:
_files = glob.glob('C:/presentations/**', recursive=True)

In [ ]:
#_files

In [ ]:
for fileName_relative in glob.glob('C:/experimental', *, recursive=True):       ## first get full file name with directores using for loop

    print("Full file name with directories: ", fileName_relative)

In [ ]:
def find_surf_and_move(src, dst, filestring):
    if not os.path.exists(dst):
        os.mkdir(dst)
    files = glob.glob(os.path.join(src,'**'), recursive=True)

    # for path, subdirs, files in os.walk(src):
    #     print(files)
    #     for name in files:
    #         if filestring in name:
    #         #if fnmatch(name, pattern):
    #             full_name = os.path.join(path, name)
    #             #destination_folder = 
    #             print(name)

                #shutil.copytree(real_name, dst)
    
    
    
    
#     files = os.listdir()
    
#     if filestring is in file:
        
#     shutil.copytree(src, dst) 

In [ ]:
find_surf_and_move(file_path, 'C:/experimental', 'cvasl')

In [ ]:
root = file_path
#pattern = "*."

for path, subdirs, files in os.walk(root):
    for name in files:
        if 'cvasl' in name:
            #if fnmatch(name, pattern):
            print(os.path.join(path, name))